In [2]:
import torch
from torch import nn
from torchdiffeq import odeint

In [3]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, future_steps=2):
        """
        Initialize the LSTM model.
        
        Args:
            input_size (int): Number of input features per time step.
            hidden_layer_size (int): Size of the LSTM hidden layer.
            future_steps (int): Number of future steps to predict.
        """
        super(LSTM_Model, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.future_steps = future_steps

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)

        # Define a fully connected layer to map from hidden state space to output space
        self.linear = nn.Linear(hidden_layer_size, future_steps)

    def forward(self, input_seq):
        """
        Forward pass through the network.
        
        Args:
            input_seq (Tensor): Input batch of sequences; shape should be (batch, seq_len, input_size).
            
        Returns:
            Tensor: The network's output with shape (batch, future_steps).
        """
        # Passing in the input into the model and obtaining outputs
        lstm_out, _ = self.lstm(input_seq)

        # Take the outputs of the last time step
        last_time_step_out = lstm_out[:, -1, :]
        print(last_time_step_out.shape)
        # Map the outputs of the last time step to the future steps
        future_predictions = self.linear(last_time_step_out)

        return future_predictions



In [4]:
class ODEFunc(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, input_size),
        )
        # 应用Xavier初始化
        self.apply(self.init_weights)

    def forward(self, t, y):
        return self.net(y)

    @staticmethod
    def init_weights(m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

class ODEBlock(nn.Module):
    def __init__(self, ode_func):
        super(ODEBlock, self).__init__()
        self.ode_func = ode_func

    def forward(self, x):
        t = torch.linspace(0., 1., 10)
        out = odeint(self.ode_func, x, t, method='dopri5')[-1]
        return out

class LSTM_ODE(nn.Module):
    def __init__(self, input_size=6, hidden_size=32, future_steps=1):
        super(LSTM_ODE, self).__init__()

        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.odefunc = ODEFunc(hidden_size, hidden_size*2)
        self.linear = nn.Linear(hidden_size, future_steps)
        
        self.odeblock = ODEBlock(self.odefunc)

        # 应用Xavier初始化
        self.apply(self.init_weights)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_time_step_out = lstm_out[:, -1, :]
        ode_output = self.odeblock(last_time_step_out)
        pred = self.linear(ode_output)
        
        return pred

    @staticmethod
    def init_weights(m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        elif type(m) == nn.LSTM:
            for param in m._flat_weights_names:
                if "weight_ih" in param:
                    torch.nn.init.xavier_uniform_(m._parameters[param])
                elif "weight_hh" in param:
                    torch.nn.init.xavier_uniform_(m._parameters[param])
                elif "bias" in param:
                    m._parameters[param].data.fill_(0.01)

In [5]:
a

NameError: name 'a' is not defined

In [6]:
a = torch.rand(128,6,1)

In [7]:
model = LSTM_ODE(1,32,1)

In [8]:
model(a).shape

torch.Size([128, 1])